In [ ]:
import os
import json

os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["http_proxy"] = "xxx"
os.environ["https_proxy"] = "xxx"

# 数据输入与提取
def load_pdf(source_files, file_):
    pdf_file = fitz.open(file_) 
    image_counter = 0
    metadata = {}
    for page_index in range(0,len(pdf_file)): 
        page = pdf_file[page_index] 
    blocks = pdf_file[page_index].get_text("blocks")
    image_meta = [ (blocks[i][4], blocks[i+1][4]) for i in range(0,len(blocks)) if blocks[i][-1]==1 ]  
    image_info= [ image_meta[0][0] if image_meta  else []] 
    image_title = [ image_meta[0][1] if image_meta else []] 
    for image in page.get_images():
        image_id = image[7]
        image_block_id = image[0] 
        image_title_block_id = image_block_id+1 
        image_dim = image[2],image[3] 
        image_counter = image_counter+1
        metadata[image_counter] = { 'page': page_index, 'image': image_id,
            'block': image_block_id,
            'image_dim': image_dim,
            'image_info': str(image_info[0]),
            'image_title': str(image_title[0]),
            'image_file': f"{image_id}_{image_block_id}.png",
            'image_path':os.path.join(source_files,f"{image_id}_{image_block_id}.png")
        }
    pix = fitz.Pixmap(pdf_file, image[0])
    pix.save(os.path.join(source_files, f"{image_id}_{image_block_id}.png"))
    with open(os.path.join(source,f'metadata.json'),'w') as f:
        json.dump(metadata,f)
        
    return metadata
    
metadata = load_pdf(source, file)

In [ ]:
# 多模态向量存储
#  pip install llama-index==0.9.48
import qdrant_client
from llama_index import ServiceContext,  SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex

client = qdrant_client.QdrantClient(path="qdrant_mm")
text_store = QdrantVectorStore(client=client, collection_name="text_collection")
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
)
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)
documents = SimpleDirectoryReader("./source_files/").load_data()
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [ ]:
# 查询与检索
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview",
    max_new_tokens=1500
)
query_engine = index.as_query_engine(multi_modal_llm=openai_mm_llm,
    similarity_top_k=3, 
    image_similarity_top_k=3
)
query_str = "What are the various stages of LLMs from pre-training"
response_mm = query_engine.query(query_str)